In [1]:
var ALS = require('eclairjs/mllib/recommendation/ALS');
var MatrixFactorizationModel = require('eclairjs/mllib/recommendation/MatrixFactorizationModel');
var Rating = require('eclairjs/mllib/recommendation/Rating');
var Tuple = require('eclairjs/Tuple');
var FloatRDD = require('eclairjs/FloatRDD');
var PairRDD = require('eclairjs/PairRDD');


In [2]:
    var SparkConf = require('eclairjs/SparkConf');
    var SparkContext = require('eclairjs/SparkContext');
    var sparkConf = new SparkConf().setAppName("Collaborative Filtering Example");
    var sc = new SparkContext(sparkConf);


 Load and parse the data


In [3]:
    var path = "examples/data/mllib/als/test.data";
    var data = sc.textFile(path);
    var ratings = data.map(function (s, Rating) {
        var sarray = s.split(",");
        return new Rating(parseInt(sarray[0]), parseInt(sarray[1]), parseFloat(sarray[2]));
    }, [Rating]);


 Build the recommendation model using ALS


In [4]:
    var rank = 10;
    var numIterations = 10;
    var model = ALS.train(ratings, rank, numIterations, 0.01);


 Evaluate the model on rating data


In [5]:
    var userProducts = ratings.map(function (r, Tuple) {
        return new Tuple(r.user(), r.product());

    }, [Tuple]);

    var predictions = PairRDD.fromRDD(model.predict(userProducts).map(function (r, Tuple) {
        return new Tuple(new Tuple(r.user(), r.product()), r.rating());
    }, [Tuple]));

    var ratesAndPreds =  PairRDD.fromRDD(ratings.map(function (r, Tuple) {
        return new Tuple(new Tuple(r.user(), r.product()), r.rating());
    }, [Tuple])).join(predictions).values();

    var MSE = FloatRDD.fromRDD(ratesAndPreds.map(function (pair) {
        var err = pair[0] - pair[1];
        return err * err;

    })).mean();

    var result = {};
    result.model = model;
    result.MSE = MSE;
    print("Mean Squared Error = " + result.MSE);


 Save and load model


In [6]:
    result.model.save(sc, "target/tmp/myCollaborativeFilter");
    var sameModel = MatrixFactorizationModel.load(sc,
        "target/tmp/myCollaborativeFilter");


In [7]:
    sc.stop();
